In [37]:
import plotly.express as px
import colorsys

def generate_colors(data, base_hue=0):
    colors = {}
    depth_map = {name: 0 for name in data['character']}
    
    # Calculate depth for each node
    for name, parent in zip(data['character'], data['parent']):
        if parent:
            depth_map[name] = depth_map[parent] + 1
    
    # Assign colors
    for i, (name, parent) in enumerate(zip(data['character'], data['parent'])):
        depth = depth_map[name]
        if parent == "":
            hue = base_hue
        else:
            if parent not in colors:
                # If parent color is not yet assigned, use a default hue
                parent_hue = base_hue
            else:
                # Convert parent color from hex to RGB, then to HLS
                parent_rgb = tuple(int(colors[parent].lstrip('#')[i:i+2], 16) for i in (0, 2, 4))
                parent_hue = colorsys.rgb_to_hls(*[c/255 for c in parent_rgb])[0] * 360
            hue_range = 120 / (depth + 1)
            hue = (parent_hue + hue_range * (i + 1) / (len(data['character']) + 1)) % 360
        
        saturation = min(50 + depth * 10, 100)
        lightness = 60 + (depth % 2) * 20 - (i % 2) * 10
        
        rgb = colorsys.hls_to_rgb(hue / 360, lightness / 100, saturation / 100)
        color = '#{:02x}{:02x}{:02x}'.format(int(rgb[0] * 255), int(rgb[1] * 255), int(rgb[2] * 255))
        
        colors[name] = color
    
    return colors

# Example data
data = dict(
    character=["Root", "Category A", "Category B", "Category C", 
               "Subcategory A1", "Subcategory A2", "Subcategory B1", "Subcategory B2",
               "Subcategory C1", "Subcategory C2"],
    parent=["", "Root", "Root", "Root", 
            "Category A", "Category A", "Category B", "Category B",
            "Category C", "Category C"],
    value=[27, 8, 6, 13, 5, 3, 4, 2, 7, 6]
)

# Generate colors
color_map = generate_colors(data)

# Create sunburst chart
fig = px.sunburst(
    data,
    names='character',
    parents='parent',
    values='value',
    color='character',
    color_discrete_map=color_map
)

fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))

# Display the figure
fig.show()

# Print color map for reference
print("\nColor map:")
for name, color in color_map.items():
    print(f"{name}: {color}")

# Print Plotly version
import plotly
print(f"\nPlotly version: {plotly.__version__}")


Color map:
Root: #cc6565
Category A: #e09584
Category B: #eabead
Category C: #e0a584
Subcategory A1: #e09751
Subcategory A2: #d88826
Subcategory B1: #e0b551
Subcategory B2: #d8ae26
Subcategory C1: #e0d251
Subcategory C2: #d8d226

Plotly version: 5.22.0
